# Bronze Layer

In [0]:
import os
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

## Yellow Taxi

In [0]:
df = spark.read.parquet("/Volumes/etl/raw/nyc_taxi/yellow/2023/yellow_tripdata_2023-01.parquet")
# .select("tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_distance", "payment_type", "fare_amount")

display(df.agg(count("*"), sum("total_amount")))
# display(df.filter(col("tpep_pickup_datetime") < "2010-01-01"))

In [0]:
# List all Parquet files in the directory
base_path = "/Volumes/etl/raw/nyc_taxi/yellow/2023/"
parquet_files = [f.path for f in dbutils.fs.ls(base_path) if f.name.endswith('.parquet')]

# Define the columns and types to standardize
columns = [
    ("VendorID", "string", "vendor_id"),
    ("RatecodeID", "string", "rate_code_id"),
    ("PULocationID", "string", "pickup_location_id"),
    ("DOLocationID", "string", "dropoff_location_id"),
    ("tpep_pickup_datetime", "timestamp", "pickup_datetime"),
    ("tpep_dropoff_datetime", "timestamp", "dropoff_datetime"),
    ("store_and_fwd_flag", "string", "store_and_fwd_flag"),
    ("payment_type", "string", "payment_type"),
    ("passenger_count", "double", "passenger_count"),
    ("trip_distance", "double", "trip_distance"),
    ("fare_amount", "double", "fare_amount"),
    ("extra", "double", "extra"),
    ("mta_tax", "double", "mta_tax"),
    ("tip_amount", "double", "tip_amount"),
    ("tolls_amount", "double", "tolls_amount"),
    ("improvement_surcharge", "double", "improvement_surcharge"),
    ("total_amount", "double", "total_amount"),
    ("congestion_surcharge", "double", "congestion_surcharge"),
    ("airport_fee", "double", "airport_fee")
]

# Read, cast, and union all files
dfs = []
for file in parquet_files:
    df = spark.read.parquet(file)
    select_exprs = [col(c[0]).cast(c[1]).alias(c[2]) for c in columns]
    dfs.append(df.select(*select_exprs))

df_standardized = dfs[0]
for df in dfs[1:]:
    df_standardized = df_standardized.unionByName(df)

df_standardized.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("etl.bronze.bronze__nyc_taxi_yellow")


In [0]:
df_bronze__nyc_taxi_yellow = spark.table("etl.bronze.bronze__nyc_taxi_yellow")

display(df_bronze__nyc_taxi_yellow.filter(trunc(col("pickup_datetime"), "month") == "2023-01-01").groupBy(trunc(col("pickup_datetime"), "month").alias("month")).agg(count("*"), sum("total_amount")))

## Green Taxi

In [0]:
df = spark.read.parquet("/Volumes/etl/raw/nyc_taxi/green/2023/green_tripdata_2023-05.parquet")
# .select("tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_distance", "payment_type", "fare_amount")

display(df)
# display(df.agg(count("*"), sum("total_amount")))
# display(df.filter(col("tpep_pickup_datetime") < "2010-01-01"))

In [0]:
[(field.name, field.dataType) for field in df.schema.fields]

In [0]:
# List all Parquet files in the directory
base_path = "/Volumes/etl/raw/nyc_taxi/green/2023/"
parquet_files = [f.path for f in dbutils.fs.ls(base_path) if f.name.endswith('.parquet')]

# Define the columns and types to standardize
columns = [
    ("VendorID", "string", "vendor_id"),
    ("RatecodeID", "string", "rate_code_id"),
    ("PULocationID", "string", "pickup_location_id"),
    ("DOLocationID", "string", "dropoff_location_id"),
    ("lpep_pickup_datetime", "timestamp", "pickup_datetime"),
    ("lpep_dropoff_datetime", "timestamp", "dropoff_datetime"),
    ("store_and_fwd_flag", "string", "store_and_fwd_flag"),
    ("payment_type", "string", "payment_type"),
    ("trip_type", "string", "trip_type"),
    ("passenger_count", "double", "passenger_count"),
    ("trip_distance", "double", "trip_distance"),
    ("fare_amount", "double", "fare_amount"),
    ("extra", "double", "extra"),
    ("mta_tax", "double", "mta_tax"),
    ("tip_amount", "double", "tip_amount"),
    ("tolls_amount", "double", "tolls_amount"),
    ("improvement_surcharge", "double", "improvement_surcharge"),
    ("total_amount", "double", "total_amount"),
    ("congestion_surcharge", "double", "congestion_surcharge"),
    ("ehail_fee", "double", "ehail_fee")
]

# Read, cast, and union all files
dfs = []
for file in parquet_files:
    df = spark.read.parquet(file)
    select_exprs = [col(c[0]).cast(c[1]).alias(c[2]) for c in columns]
    dfs.append(df.select(*select_exprs))

df_standardized = dfs[0]
for df in dfs[1:]:
    df_standardized = df_standardized.unionByName(df)

df_standardized.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("etl.bronze.bronze__nyc_taxi_green")


In [0]:
df_bronze_green = spark.table("etl.bronze.bronze__nyc_taxi_green")

display(df_bronze_green)